In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

DATA = 'data/'

In [2]:
train_df = pd.read_csv(DATA + 'sales_train.csv.gz')
test_df = pd.read_csv(DATA + 'test.csv.gz')

train_df.date = pd.to_datetime(train_df.date, format='%d.%m.%Y')

train_df.head()

In [4]:
def select_month(month, year, data):
    mask = (data.date.dt.month == month) & (data.date.dt.year == year) & (
        data.item_cnt_day.between(0, 20))
    return data[mask]

In [9]:
october_2015 = select_month(10, 2015, train_df)

test_df['shop_item'] = test_df.shop_id.astype(str) + "_" + test_df.item_id.astype(str)
october_2015['shop_item'] = october_2015.shop_id.astype(str) + "_" + october_2015.item_id.astype(str)

C:\Users\kipil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
grouped = october_2015.groupby('shop_item').item_cnt_day.mean().to_frame().reset_index()

df = test_df.merge(grouped, how='left', on='shop_item')

In [12]:
df.shape
test_df.shape
df.head()

,ID,shop_id,item_id,shop_item,item_cnt_day
0,0,5,5037,5_5037,NaN
1,1,5,5320,5_5320,NaN
2,2,5,5233,5_5233,1.0
3,3,5,5232,5_5232,NaN
4,4,5,5268,5_5268,NaN


In [ ]:
submissions_df = pd.read_csv(DATA + "sample_submission.csv.gz")

In [13]:
submissions_df.item_cnt_month = df.item_cnt_day.fillna(0)
submissions_df.to_csv('submissions/previous_value_benchmark_manual_last_month_with_0.csv', index=False)

In [15]:
submissions_df.item_cnt_month = df.item_cnt_day.fillna(1)
submissions_df.to_csv('submissions/previous_value_benchmark_manual_last_month_with_1.csv', index=False)